# Lab4 - edit distance, LCS, diff

## Edit distance

In [1]:
# no change = 0
# insert = 1
# remove = 2
# replace = 3


def edit_dist(text1, text2):
    n = len(text1)
    m = len(text2)
    
    dp = [[0]*(n+1) for _ in range(m+1)]
    op = [[0]*(n+1) for _ in range(m+1)]
    op[1][0] = 2
    for i in range(n+1):
        dp[0][i]=i
    for i in range(m+1):
        dp[i][0] = i
    
    
    for i in range(1,m+1):
        for j in range(1,n+1):
            if text1[j-1] != text2[i-1]:
                insert = dp[i][j-1] + 1
                remove = dp[i-1][j] + 1
                replace = dp[i-1][j-1] + 1
                dp[i][j] = min(insert, remove, replace)
                if dp[i][j] == replace:
                    op[i][j] = 3
                elif dp[i][j] == remove:
                    op[i][j] = 2
                else:
                    op[i][j] = 1
                
            else:
                dp[i][j] = dp[i-1][j-1]

    
    i = m
    j = n
    curr = list(text2)
    for l in curr:
        print(l, end="")
    print()
    while i>=0 and j>=0:
        if op[i][j] == 0:
            i -=1
            j -=1
            continue
        elif op[i][j] == 1:
            curr.insert(i, text1[j-1])
            for k in range(i):
                print(curr[k], end="")
            print("[>", end="")
            print(curr[k], end="")
            print("<]", end="")
            k+=1
            while k<len(curr):
                print(curr[k], end="")
                k+=1
            
            j -=1
        elif op[i][j] == 3:
            prev = curr[i-1]
            curr[i-1] = text1[j-1]
            k =0
            while k < i-1: 
                print(curr[k], end="")
                k+=1
            print("[", end="")
            print(prev, end="")
            print("->", end="")
            print(curr[k], end="")
            print("]", end="")
            k+=1
            while k<len(curr):
                print(curr[k], end="")
                k+=1
            i -=1
            j -=1
        else:
            curr.pop(i-1)
            for k in range(i):
                print(curr[k], end="")
            print("[*", end="")
            print(curr[k], end="")
            print("*]", end="")
            k+=1
            while k<len(curr):
                print(curr[k], end="")
                k+=1
            
            i -=1
        print()
    for l in curr:
        print(l, end="")
    print()
    
    return dp[m][n]
    

In [2]:
print(edit_dist("abcd", "cdab"))

cdab
cda[b->d]
cd[a->c]d
c[d->b]cd
[c->a]bcd
abcd
4


In [3]:
print(edit_dist("los", "kloc"))

kloc
klo[c->s]
l[*l*]os
los
2


In [4]:
print(edit_dist("Łódź", "Lodz"))

Lodz
Lod[z->ź]
L[o->ó]dź
[L->Ł]ódź
Łódź
3


In [5]:
print(edit_dist("kwintesencja", "quintessence"))

quintessence
quintessenc[e->a]
quintessenc[>c<]ja
quintes[*s*]encja
q[u->w]intesencja
[q->k]wintesencja
kwintesencja
5


In [6]:
print(edit_dist("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG"))

ATGAGGCTCTGGCCCCTG
ATGAGGCTCTGGCCCCT[>T<]CG
ATGAGGCTCTGGC[C->G]CCTCG
ATGAGGCTCTG[G->C]CGCCTCG
ATGAGGCTCT[G->A]CCGCCTCG
ATGAGGCTT[*T*]ACCGCCTCG
ATGAG[G->T]CTTACCGCCTCG
ATGA[G->A]TCTTACCGCCTCG
ATGAATCTTACCGCCTCG
7


## LCS

In [7]:
def lcs(x,y):
    return (len(x) + len(y) - edit_dist(x, y))/2

In [8]:
def backtrack_lcs(dp, text1, text2, i, j):
    if i<0 or j<0:
        return ""
    if text1[i] == text2[j]:
        return backtrack_lcs(dp, text1, text2, i-1, j-1) + text1[i]
    if dp[i][j-1] > dp[i-1][j]:
        return backtrack_lcs(dp, text1, text2, i, j-1)
    return backtrack_lcs(dp, text1, text2, i-1, j)

In [9]:
def LCS(text1, text2, show = True, ret = False):
    n = len(text1)
    m = len(text2)
    
    dp = [ [0]*(m+1) for _ in range(n+1)]
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            if text1[i-1] == text2[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i][j-1], dp[i-1][j])
    
    if show:
        print(backtrack_lcs(dp, text1, text2, n-1, m-1))
    if ret:
        return dp
    
    return dp[n][m]
    
    

In [10]:
LCS("abcd", "cdab")

ab


2

In [11]:
LCS("kwintesencja", "quintessence")

intesenc


8

## Tokenization

In [12]:
from spacy.language import Language
from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab
import numpy as np

f = open("romeo-i-julia-700.txt")
text = f.read()
tokenizer = Tokenizer(Language(Vocab()).vocab)
tokens = tokenizer(text)
f.close()

text1 = []
text2 = []

for t in tokens:
    if np.random.random() >= 0.03:
        text1.append(t)
    if np.random.random() >= 0.03:
        text2.append(t)

print(len(tokens))
print(len(text1))
print(len(text2))
print(LCS(text1, text2, False))




2272
2196
2197
2123


## Diff

In [13]:
def diff(text1, text2):
    dp = LCS(text1, text2, False, True)
    i = len(text1) - 1
    j = len(text2) - 1
    ans = []
    while i >= 0 or j >= 0:
        if text1[i] == text2[j]:
            i -= 1
            j -= 1
        elif i >= 0 and dp[i][j-1] < dp[i-1][j]:
            ans.append(" < " +  str(i) + " " + text1[i])
            i -= 1
        elif j >= 0:
            ans.append(" > " +  str(j) + " " + text2[j])
            j -= 1
    ans.reverse()
    for line in ans:
        print(line)
    

In [14]:
f = open("romeo-i-julia-700.txt")
text = f.read()
tokenizer = Tokenizer(Language(Vocab()).vocab)
text = text.split('\n')
f.close()

text1 = []
text2 = []
separator = " "

for line in text:
    tokens = tokenizer(line)
    t1 = []
    t2 = []
    for token in tokens:
        if np.random.random() >= 0.03:
            t1.append(str(token))
        if np.random.random() >= 0.03:
            t2.append(str(token))
    text1.append(separator.join(t1) + '\n')
    text2.append(separator.join(t2) + '\n')
    

print(len(text1))
print(len(text2))
print(LCS(text1, text2, False))




701
701
614


In [15]:
diff(text1, text2)

 < 10   * ESKALUS — panujący w Weronie

 > 10 * ESKALUS — książę panujący w Weronie

 < 14   * ROMEO syn Montekiego

 > 14   * ROMEO — syn Montekiego

 < 15   * MERKUCJO — krewny księcia

 < 16   * BENWOLIO — synowiec Montekiego

 > 15   * MERKUCJO — krewny księcia

 > 16   * BENWOLIO — Montekiego

 < 21   * GRZEGORZ — słudzy Kapuleta

 > 21   * SAMSON, GRZEGORZ — słudzy Kapuleta

 < 22   * ABRAHAM — służący Montekiego

 < 23   * APTEKARZ

 > 22   * ABRAHAM — służący Montekiego

 > 23   *

 < 26   * PIOTR

 > 26 * PIOTR

 < 27   * DOWÓDCA WARTY

 < 28 * PANI — małżonka Montekiego

 > 27   * DOWÓDCA WARTY

 > 28   * PANI MONTEKI — małżonka Montekiego

 < 31   * MARTA mamka Julii

 > 31   * MARTA — mamka Julii

 < 32   * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i inne osoby.

 > 32   * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i

 < 37 Rzecz odbywa się przez wi